<span style="font-size:36px"><b>Baseline Model Tutorial</b></span>

Copyright &copy; 2020 Gunawan Lumban Gaol

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language overning permissions and limitations under the License.

# Import Packages

In [1]:
import os
import sys
import glob

import numpy as np
import pandas as pd
from IPython.core.display import display, HTML
import tensorflow as tf
import tensorflow.keras.backend as K
print("tf.__version__ = "+tf.__version__)

from gurih.data.data_generator import DataGenerator
from gurih.models.model import BaselineASRModel
from gurih.models.model_utils import CharMap, ctc_decode, wer

%load_ext autoreload
%autoreload 2

tf.__version__ = 2.1.0


# Train Model

Define constant for ASR Model.

In [2]:
CHAR_TO_IDX_MAP = CharMap.CHAR_TO_IDX_MAP
IDX_TO_CHAR_MAP = CharMap.IDX_TO_CHAR_MAP

MAX_SEQ_LENGTH = 3000
MAX_LABEL_LENGTH = 300
BATCH_SIZE = 16
MAX_EPOCHS = 5000

Create and compile keras model.

In [3]:
BaselineASR = BaselineASRModel(input_shape=(MAX_SEQ_LENGTH, 39), vocab_len=len(CharMap()))
BaselineASR.compile()

Model directory is set to ../../models/
Documentation directory is set to ../../docs/

Model: "BaselineASR_f200_k11_s2_pvalid_nlstm200_ndense29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, 3000, 39)]        0         
_________________________________________________________________
masking (Masking)            (None, 3000, 39)          0         
_________________________________________________________________
conv1 (Conv1D)               (None, 1495, 200)         86000     
_________________________________________________________________
bidirectional (Bidirectional (None, 1495, 400)         641600    
_________________________________________________________________
the_output (TimeDistributed) (None, 1495, 30)          12030     
Total params: 739,630
Trainable params: 739,630
Non-trainable params: 0
_________________________________________________________

Get model output sequence length for ctc input. This is required as we are using custom `Lambda` layer inside model.

In [4]:
CTC_INPUT_LENGTH = BaselineASR.model.get_layer('the_output').output.shape[1]
CTC_INPUT_LENGTH

1495

## Train using `fit_generator()`

Prepare data using `DataGenerator`.

In [5]:
train_dir = "../../dataset/interim/Model-010a/train/"
valid_dir = "../../dataset/interim/Model-010a/valid/"
test_dir = "../../dataset/interim/Model-010a/test/"

Directories must contain all `.npz` and `.txt` files required for `DataGenerator`.

In [6]:
train_generator = DataGenerator(input_dir=train_dir,
                                max_seq_length=MAX_SEQ_LENGTH,
                                max_label_length=MAX_LABEL_LENGTH,
                                ctc_input_length=CTC_INPUT_LENGTH,
                                char_to_idx_map=CHAR_TO_IDX_MAP,
                                batch_size=BATCH_SIZE)

validation_generator = DataGenerator(input_dir=valid_dir,
                                     max_seq_length=MAX_SEQ_LENGTH,
                                     max_label_length=MAX_LABEL_LENGTH,
                                     ctc_input_length=CTC_INPUT_LENGTH,
                                     char_to_idx_map=CHAR_TO_IDX_MAP,
                                     batch_size=BATCH_SIZE)

Fit model. Pass validation generator if using `EarlyStopping` and `ModelCheckpoint` callbacks.

In [7]:
BaselineASR._callbacks(min_delta=1e-4, patience=20, save_weights_only=True)

Optionally, retrained pretrained model.

In [8]:
# filename = os.path.abspath("../../models/BaselineASR_f200_k11_s2_pvalid_nlstm200_ndense29.h5") # must provide abs path
# BaselineASR.load(filename)

In [9]:
BaselineASR.fit_generator(train_generator=train_generator,
                          validation_generator=validation_generator,
                          epochs=MAX_EPOCHS,
#                           use_multiprocessing=False,
#                           worker=4,
)

Instructions for updating:
Please use Model.fit, which supports generators.
  {'ctc': '...'}
    to  
  ['...']
  {'ctc': '...'}
    to  
  ['...']
Train for 37 steps, validate for 25 steps
Epoch 1/5000
37/37 [==============================] - 399s 11s/step - loss: 1024.6350 - val_loss: 722.6433
Epoch 2/5000
37/37 [==============================] - 432s 12s/step - loss: 697.7732 - val_loss: 712.9141
Epoch 3/5000
37/37 [==============================] - 418s 11s/step - loss: 691.3023 - val_loss: 704.4966
Epoch 4/5000
37/37 [==============================] - 427s 12s/step - loss: 673.8215 - val_loss: 678.7964
Epoch 5/5000
37/37 [==============================] - 436s 12s/step - loss: 623.4852 - val_loss: 627.5742
Epoch 6/5000
37/37 [==============================] - 434s 12s/step - loss: 549.1934 - val_loss: 555.2151
Epoch 7/5000
37/37 [==============================] - 435s 12s/step - loss: 484.5785 - val_loss: 505.4704
Epoch 8/5000
37/37 [==============================] - 441s 12s/step

KeyError: 'loss'

See training plots.

In [43]:
BaselineASR.plot_history()

AttributeError: Model is not fitted. No history figure found. Call fit() method first

In [25]:
BaselineASR.save()

Model BaselineASR_f200_k11_s2_pvalid_nlstm200_ndense29 saved at: ../../models/BaselineASR_f200_k11_s2_pvalid_nlstm200_ndense29.json
Weights serialized at: ../../models/BaselineASR_f200_k11_s2_pvalid_nlstm200_ndense29.h5


## Evaluation

Evaluate using test data.

In [70]:
model = BaselineASR

Alternatively, load pretrained model.

In [ ]:
# model = BaselineASRModel(input_shape=(3000, 39), vocab_len=29)
# model.compile()

# filename = os.path.abspath("../../docs/Model010b/BaselineASR_f200_k11_s2_pvalid_nlstm200_ndense29.h5") # must provide abs path
# model.load(filename)

In [71]:
test_generator = DataGenerator(input_dir=test_dir,
                                max_seq_length=MAX_SEQ_LENGTH,
                                max_label_length=MAX_LABEL_LENGTH,
                                ctc_input_length=CTC_INPUT_LENGTH,
                                char_to_idx_map=CHAR_TO_IDX_MAP,
                                batch_size=3,
                                shuffle=False) # set to False to ensure correct alignment between y_true and y_pred

X_test = test_generator[0][0]['the_input']
X_test.shape

(3, 3000, 39)

In [72]:
ctc_matrix = model.predict(X_test)
ctc_matrix.shape

(3, 1495, 30)

Calculate and display word error rate.

In [73]:
def get_y(filename):
    with open(filename, 'r') as f:
        return f.readline()

In [74]:
txts = sorted(glob.glob(test_dir+"*.txt"))

y_true = [get_y(txt) for txt in txts]
y_pred = ctc_decode(ctc_matrix, IDX_TO_CHAR_MAP, greedy=False)

In [75]:
word_error_rate = wer(y_true, y_pred)
print('WER %.4f %%' % word_error_rate)

WER 91.4696 %


Display prediction errors. 
* Red colors are <span style="background-color:rgba(255, 0, 0, 0.23)">missing word</span>
* Green colors are <span style="background-color:rgba(0, 128, 0, 0.23)">insertion word</span>.
* Yellow colors are <span style="background-color:rgba(255, 165, 0, 0.5)">substitution word</span> and the references are in parenthesis.

In [76]:
_ = wer(y_true, y_pred, write_html=False)

In [77]:
with open("diff.html", 'r') as f:
    html = f.readlines()[0]
display(HTML(html))

In [78]:
for ctc in ctc_matrix:
    print(ctc.shape)
    tmp = np.argmax(ctc, axis=1)
    out_str = ''.join([IDX_TO_CHAR_MAP[x] for x in tmp])
    print(out_str)
    break

(1495, 30)
daaa%mm%%%%%uuu%%lu%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% %%% %de%m%%%%%tuu%%%%%%h%%%%%%%%j%%aaa%%%%llaa%%mm%%%%%%%%ke%%%lii%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%  %a%%%%%n%%%%aan%%%%%%%%%%%%%  yee%%%%duu%%%%%%% %mee%%%%%%%%%t%aa%n%%%a%%%%%%muu%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%   %a%%%%%%lmu%% %%%%ss%ae%mm%%ppaaii%%g%%%%see%%%%%aaa%%%%%% iii%%%%muue%%%%%%%daann%%a%%%%%m%u%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%  %aa%%%%kka%%%bee%%%%%%%%%kii%e%%% mm%a%%%%%%%l%a%%%%  %meeree%%%%ka%% dii%%%%see%%%%%%%daaanngg%%%  %%%ssu%%n%%ee%%%%a%%%%%%%%%%%o%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%  ienng%a%ng%%%% piii%%%%d%aa%%%  meen%%%%%%ssu%%%%%%%dd%a%%rra%%%%m%%%%%%%a%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% %%sse%%%%aa%%%%%k%%%%s%en%%%%bleeee%%%% %%%aa%%%%%%%%%%%oann%%%%  ii%%%%%mu%%%%llaa%%%%%  p%a%%%%%%%%%p%aa%%%%%sse%%%d%e%%%rra%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% %%l%aa%%%%ll%%

In [79]:
y_true

['kemudian mereka berjalan melalui padang gurun, menempuh jalan keliling tanah edom dan tanah moab, lalu sampai ke sebelah timur tanah moab, maka berkemahlah mereka di seberang sungai arnon, dengan tidak masuk daerah moab, sebab sungai arnon itulah batas daerah moab.',
 'beberapa waktu kemudian, dalam musim menuai gandum, pergilah simson mengunjungi isterinya, dengan membawa seekor anak kambing, serta berkata aku mau ke kamar mendapatkan isteriku. tetapi ayah perempuan itu tidak membiarkan dia masuk.',
 'berkatalah orang filistin siapakah yang melakukan ini orang menjawab simson, menantu orang timna itu, sebab orang itu telah mengambil isteri simson dan memberikannya kepada kawannya. kemudian pergilah orang filistin ke sana dan membakar perempuan itu beserta ayahnya.']

In [80]:
y_pred

['dambulu emtuh jalam keli anan yedu metanamu alu sampai sela imuedanamu akabekie mala mereka diselang suneamo engag pida mensu daramua sebaksenle anon imula papa sederau lalorang i sere gnmiri tusanke bemdesikoa rakea er amua ra i isio.',
 'buberapawa tu kemudian dalahmusi menuwaikandu perdilah sinm son memuntugi is terinya diban membaoas e kur anakampi serta berka a kembekana nakaik etapi a ya perempuan itu tida memakan ia mas',
 'berkatalah orang tilisti canlapkeaeamaukadi orang menjan sierisan nha tiora tingaituba antitelan anya iseisinso an menbikan ya pada kana kemudian berdila orang tilis tinm gesana dan membakarepermpua itu r a alh']